# Load a pytorch model and make it into an onnx

In [1]:
import torch
print(torch.__version__)

import onnxscript
print(onnxscript.__version__)

import onnxruntime
print(onnxruntime.__version__)

import onnxruntime_extensions
print(onnxruntime_extensions.__version__)

2.6.0
0.2.5
1.22.0


/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.14.0


In [2]:
from miditok import REMI

t_conf_path = "trained_tokenizer.json"
tokenizer = REMI(params=t_conf_path)
print(tokenizer.vocab_model)

{'>\x87\x95': 5138, 'Y\x86\x93': 4087, 'ĆĜ': 4283, '<\x87\x8e': 3372, 'b\x8b\x8f': 6774, '\x88°': 3414, '\x85\x95M\x85\x95': 6041, '\x89É': 3685, 'Q\x8d\x90': 2693, 'ÿ^': 6875, 'Q\x8d\x94': 4455, 'íQ': 3801, 'ï7': 7205, '_\x83\x8f': 4938, 'X\x8a\x95': 2622, '>\x84\x8f': 2939, 'V\x8c\x91': 5526, 'Ĳ\x86\x90': 7942, '\x8b¨': 2688, 'D\x81\x8f': 5088, '\x84\x8fĊ': 4933, 'éC': 5550, 'L\x87\x91': 1480, 'ė\x87\x8f': 3581, 'H\x84\x94': 5034, '\\\x88\x94': 6900, 'Ê': 169, 'Ĥ\x8d\x8e': 5193, 'I\x8b\x91': 5507, 'M\x88\x92': 2621, 'l\x84\x8e': 7699, 'û': 218, ':\x86\x8e': 3651, '_\x86\x95': 7203, '\x8a\x8eú': 3391, 'æ6': 1890, 'ÿY': 3907, 'ė\x8a\x91': 1394, 'S\x8d\x9d': 7165, 'ù7': 6620, 'j\x86\x90': 4726, 'ę\x86\x8e': 2240, 'E\x8a\x8f': 4074, '\x86\x8fê': 3765, '\x8a\x94M\x8a\x94': 6178, '\x8b\x90ê': 3184, '\x7f\x91': 926, '\x83Ç': 7684, 'ĸ\x82\x91': 2046, 'c\x86\x91': 3955, 'îė\x8a\x8f': 7206, 'K\x84\x8e': 1575, '\x87Æ': 4699, 'T\x8d\x93': 7674, '\x9f': 126, 'ĝ\x84\x8e': 536, '[\x8b\x95': 3992, '

In [3]:
import torch
import torch.nn as nn
from transformers import TransfoXLLMHeadModel
import json

class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, mems):
        mems = list(torch.unbind(mems, dim = 0))
        pred, mems = self.model(input_ids = input_ids, mems = mems)
        mems = torch.stack(mems, dim = 0)
        return pred, mems

model = TransfoXLLMHeadModel.from_pretrained("./best_model", torchscript=True)
config_path = "./best_model/config.json"
with open(config_path, "r") as f:
    conf = json.load(f)

model.eval()
print(model.transformer.mem_len)
model_wrapper = ModelWrapper(model)    

TransfoXLLMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of TransfoXLLMHeadModel were not initialized from the model checkpoint at ./best_model and are newly initialized: ['crit.out_layers.0.weight']
You

512


In [ ]:
tokenized_input = tokenizer.encode("sample.midi")[0]
ids = tokenized_input.ids
ids_cut = ids[0:50]
ids_tensor = torch.tensor(ids_cut).unsqueeze(0)

model_wrapper.eval()
torch.manual_seed(42)
gen = []

init_mems = torch.zeros(conf["n_layer"], conf["mem_len"], 1, conf["d_model"], dtype=torch.float32)
print("Initial mems shape:", init_mems.shape)
pred, mems = model_wrapper(ids_tensor, init_mems)
token = torch.argmax(pred, dim=-1)[:,-1]
gen.append(token.item())
pred, mems = model_wrapper(token.unsqueeze(0), mems)
token = torch.argmax(pred, dim=-1)[:,-1]
gen.append(token.item())
print(f"generated tokens: {gen}")

Initial mems shape: torch.Size([8, 512, 1, 256])
tensor(562)
tensor(50)
tensor(513)
tensor(1)
generated tokens: [231, 4241]


In [4]:
from torch.export import Dim

ids_tensor = torch.tensor(ids[:1000]).unsqueeze(0)
export_mems = mems
export_to_onnx = torch.onnx.export(model_wrapper,
                                (ids_tensor, export_mems),
                                "model.onnx",
                                input_names = ["input_ids", "mems"],
                                output_names = ["pred_score"],
                                    dynamic_axes={"input_ids": {1: "sequence"},
                                    "pred_score": {1: "sequence"}},
                                #strict = True,
                                #dynamo = True,
                                optimize=True
                                )

NameError: name 'ids' is not defined

In [ ]:
import onnx
onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)
print(onnx_model.graph.node)

[input: "model.transformer.word_emb.emb_layers.0.weight"
output: "model.crit.out_layers.0.weight"
name: "Identity_518"
op_type: "Identity"
, output: "/Constant_output_0"
name: "/Constant"
op_type: "Constant"
attribute {
  name: "value"
  t {
    dims: 8
    data_type: 7
    raw_data: "\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000\001\000\000\000\000\000\000\000"
  }
  type: TENSOR
}
, input: "mems"
input: "/Constant_output_0"
output: "/Split_output_0"
output: "/Split_output_1"
output: "/Split_output_2"
output: "/Split_output_3"
output: "/Split_output_4"
output: "/Split_output_5"
output: "/Split_output_6"
output: "/Split_output_7"
name: "/Split"
op_type: "Split"
attribute {
  name: "axis"
  i: 0
  type: INT
}
, output: "/Constant_1_output_0"
name: "/Constant_1"
op_type: "Constant"
attribute {
  name: "value"
  

In [21]:
import onnxruntime as ort
gen = []
ids_cut = ids[:2]  # Ensure the input is of the correct length
ids_tensor = torch.tensor(ids_cut, dtype = torch.int64).unsqueeze(0)
mems = torch.zeros(conf["n_layer"], conf["mem_len"], 1, conf["d_model"], dtype=torch.float32)

ort_sess = ort.InferenceSession("model.onnx", providers=['CPUExecutionProvider'])

out = ort_sess.run(None, {"input_ids": ids_tensor.numpy(), "mems": mems.numpy()})
pred, mems_first = out
token = torch.argmax(torch.tensor(pred), dim=-1)[:,-1]
gen.append(token.item())
#add a dummy token to the beginning of the input to work with onnx
out, mems_second = ort_sess.run(None, {"input_ids": token.unsqueeze(0).numpy(), "mems": mems_first})
token = torch.argmax(torch.tensor(out), dim=-1)[:,-1]
gen.append(token.item())


In [23]:
print(mems_first[0, -3, 0, :])
print(mems_second[0, -3, 0, :])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-5.40248811e-01  1.60691428e+00 -1.73486400e+00 -6.55573606e-01
 -4.92238104e-01 -2.36602008e-01  5.74063540e-01  4.01358992e-01
  2.25293413e-01 -9.27107215e-01  1.29422569e+00 -1.27036572e-02
  1.86793268e-01 -2.36996

In [8]:
exported_program = torch.export.export(model_wrapper,
                                       (ids_tensor, export_mems))
exported_program.module().forward(ids_tensor, export_mems)

Unsupported: Dynamic slicing on data-dependent value is not supported

from user code:
   File "/var/folders/41/_0pxp7tj00z70s4v6p77bl9w0000gn/T/ipykernel_9440/3723646833.py", line 13, in forward
    pred, mems = self.model(input_ids = input_ids, mems = mems)
  File "/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/transformers/models/deprecated/transfo_xl/modeling_transfo_xl.py", line 1196, in forward
    transformer_outputs = self.transformer(
  File "/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/transformers/models/deprecated/transfo_xl/modeling_tr

```
tensor([[[-16.4395, -20.0806,  -8.5010,  ..., -10.4721, -12.0833, -12.1338],
         [-16.3543, -22.9913,  -7.8673,  ..., -11.3057, -12.5514, -11.4206],
         [-18.0565, -26.1898,  -9.9663,  ..., -15.4819, -14.0376, -11.6159],
         ...,
         [-19.8914, -27.6164, -10.8207,  ...,  -9.7397, -12.0389, -10.3458],
         [-18.2326, -25.8786, -11.9248,  ..., -11.8850, -11.2620, -13.3507],
         [-17.8503, -24.7115,  -7.6854,  ...,  -9.3375,  -5.6981,  -5.6824]]],
       grad_fn=<ViewBackward0>)
```

In [9]:
import torch
import torch.nn as nn
from torch.export import export, Dim

mems_dummy = torch.zeros(conf["n_layer"], conf["mem_len"], 1, conf["d_model"], dtype=torch.float32)
#fill mems with random values
mems_dummy = torch.randn_like(mems_dummy)  # Random initialization for tracing
input_ids_dummy = torch.tensor(ids[:1000], dtype=torch.int64).unsqueeze(0)  # Dummy input_ids for tracing

class GeneratorWrapper(nn.Module):
    def __init__(self):
        super().__init__()
        
        with torch.no_grad():
            self.model = torch.jit.trace(model_wrapper.eval(), 
                                (ids_tensor, mems_dummy),
                                optimize=True)

           
        self.batch_size = 1
        self.n_layer = model_wrapper.model.transformer.n_layer
        self.mem_len = model_wrapper.model.transformer.mem_len
        self.d_model = model_wrapper.model.transformer.d_model
        self.vocab_size = model_wrapper.model.transformer.n_token

        #init mems
        self.init_mems = torch.zeros(self.n_layer, self.mem_len, self.batch_size, self.d_model, dtype=torch.float32)

        #Create tracker for repetition penalty
        self.token_counts = torch.zeros(self.batch_size, self.vocab_size, dtype=torch.int32)

    def _rep_penalty(self, logits, rep_penalty: float):
        penalty_mask = self.token_counts > 0
        logits = logits.clone()
        logits[penalty_mask] /= rep_penalty
        return logits

    def _top_k_sampling(self, logits, top_k: int):
        """Apply top-k filtering to logits"""
        if top_k > 0:
            # Get top-k values and indices
            top_k = min(top_k, logits.size(-1))
            top_k_values, _ = torch.topk(logits, top_k, dim=-1)
            # Set all values below top-k to -inf
            logits[logits < top_k_values[:, -1]] = float('-inf')
        return logits

    def _sample_token(self, logits, top_k: int):
        logits = self._top_k_sampling(logits, top_k)
        probabilities = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probabilities, num_samples=1).squeeze(-1)
        return next_token
    
    def forward(self, input_ids, 
                n_events:int, 
                temperature: float, 
                repetition_penalty: float,
                topk: int,
                ):
        
        generated_tokens = torch.zeros(self.batch_size, n_events, dtype=torch.long)

        # Initial prediction
        pred, mems = self.model(input_ids, self.init_mems) # [batch_size, seq_len, vocab_size]
        pred_logits = pred[:, -1, :] # Get logits for the last token

        for step in range(n_events):
            scaled_logits = pred_logits / temperature

            scaled_logits = self._rep_penalty(scaled_logits, repetition_penalty)

            next_token = self._sample_token(scaled_logits, topk)
            #next_token = torch.argmax(scaled_logits, dim=-1)
            generated_tokens[:, step] = next_token  
            batch_indices = torch.arange(self.batch_size)
            self.token_counts[batch_indices, next_token] += 1

            # Predict next logits, adding new token to input sequence
            pred, mems = self.model(next_token.unsqueeze(0), mems)  # Keep predicting for new sequence
            pred_logits = pred[:, -1, :]  # Get logits for the last token

        return generated_tokens

gen_wrapper = GeneratorWrapper()
gen_wrapper.eval()

/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: FutureWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution()` instead
  return fn(*args, **kwargs)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/transformers/models/deprecated/transfo_xl/modeling_transfo_xl.py:1020: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask_len_tensor = torch.tensor(mask_len) #if isinstance(mask_len, int) else mask_len.clone().detach()
/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packa

GeneratorWrapper(
  (model): ModelWrapper(
    original_name=ModelWrapper
    (model): TransfoXLLMHeadModel(
      original_name=TransfoXLLMHeadModel
      (transformer): TransfoXLModel(
        original_name=TransfoXLModel
        (word_emb): AdaptiveEmbedding(
          original_name=AdaptiveEmbedding
          (emb_layers): ModuleList(
            original_name=ModuleList
            (0): Embedding(original_name=Embedding)
          )
          (emb_projs): ParameterList(original_name=ParameterList)
        )
        (drop): Dropout(original_name=Dropout)
        (layers): ModuleList(
          original_name=ModuleList
          (0): RelPartialLearnableDecoderLayer(
            original_name=RelPartialLearnableDecoderLayer
            (dec_attn): RelPartialLearnableMultiHeadAttn(
              original_name=RelPartialLearnableMultiHeadAttn
              (qkv_net): Linear(original_name=Linear)
              (drop): Dropout(original_name=Dropout)
              (dropatt): Dropout(origi

In [51]:
generated = gen_wrapper(ids_tensor, n_events = torch.tensor(10), 
                          temperature = torch.tensor(1.2), 
                          repetition_penalty = torch.tensor(0.9),
                          topk = torch.tensor(10)
                          )

In [52]:
#export
n_events = torch.tensor(10, dtype=torch.int64)
temperature = torch.tensor(1.2, dtype=torch.float32)
repetition_penalty = torch.tensor(0.9, dtype=torch.float32)
topk = torch.tensor(10, dtype=torch.int64)
with torch.no_grad():
    gen_model = torch.jit.script(gen_wrapper,
                    (ids_tensor, n_events, temperature, repetition_penalty))
gen_model.eval()
gen_model(ids_tensor, n_events, temperature, repetition_penalty, topk)

/var/folders/41/_0pxp7tj00z70s4v6p77bl9w0000gn/T/ipykernel_9080/1168489218.py:7: FutureWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution()` instead
  gen_model = torch.jit.script(gen_wrapper,


tensor([[ 196, 3379,  203,   60,  461,  212,   62,  475,   22,  521]])

In [217]:
with torch.no_grad():
    torch.onnx.export(gen_model,
                    (ids_tensor, n_events, temperature, repetition_penalty, topk),
                    "gen.onnx",
                    input_names = ["input_ids", "n_events", "temperature", "repetition_penalty", "topk"],
                    output_names = ["gen_seq"],
                    dynamic_axes = {"input_ids": {1: "sequence"},
                                    "gen_seq": {1: "sequence"}},
                    do_constant_folding=True,
                    dynamo=True,
                    optimize=True
                    )

/Users/mikaelhokkanen/anaconda3/envs/midi-transformer/lib/python3.10/site-packages/torch/onnx/_internal/exporter/_compat.py:262: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  warnings.warn(


[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with `torch.export.export`...
[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with `torch.export.export`... ❌
[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with Torch Script...


W0528 12:34:19.862000 2531 site-packages/torch/fx/experimental/symbolic_shapes.py:6307] failed during evaluate_expr(u1, hint=None, size_oblivious=False, forcing_spec=False
E0528 12:34:19.863000 2531 site-packages/torch/fx/experimental/recording.py:299] failed while running evaluate_expr(*(u1, None), **{'fx_node': False})
W0528 12:34:19.885000 2531 site-packages/torch/_dynamo/eval_frame.py:343] could not determine __code__ for RecursiveScriptModule(
W0528 12:34:19.885000 2531 site-packages/torch/_dynamo/eval_frame.py:343]   original_name=GeneratorWrapper
W0528 12:34:19.885000 2531 site-packages/torch/_dynamo/eval_frame.py:343]   (model): ModelWrapper(
W0528 12:34:19.885000 2531 site-packages/torch/_dynamo/eval_frame.py:343]     original_name=ModelWrapper
W0528 12:34:19.885000 2531 site-packages/torch/_dynamo/eval_frame.py:343]     (model): TransfoXLLMHeadModel(
W0528 12:34:19.885000 2531 site-packages/torch/_dynamo/eval_frame.py:343]       original_name=TransfoXLLMHeadModel
W0528 12:34:

[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with Torch Script... ❌
[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with internal Dynamo apis...


W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343] could not determine __code__ for RecursiveScriptModule(
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]   original_name=GeneratorWrapper
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]   (model): ModelWrapper(
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]     original_name=ModelWrapper
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]     (model): TransfoXLLMHeadModel(
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]       original_name=TransfoXLLMHeadModel
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]       (transformer): TransfoXLModel(
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]         original_name=TransfoXLModel
W0528 12:34:20.602000 2531 site-packages/torch/_dynamo/eval_frame.py:343]         (word_emb): AdaptiveEmbedding(


[torch.onnx] Obtain model graph for `RecursiveScriptModule([...]` with internal Dynamo apis... ❌


TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and summit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'ValueError'>: Exporting a ScriptModule is not supported. Maybe try converting your ScriptModule to an ExportedProgram using `TS2EPConverter(mod, args, kwargs).convert()` instead.

(Refer to the full stack trace above for more information.)

In [206]:
#Check onnx model
onnx_gen_model = onnx.load("gen.onnx")
onnx.checker.check_model(onnx_gen_model)
print(onnx_gen_model.graph.node)

[input: "input_ids"
output: "/model/model/Shape_1_output_0"
name: "/model/model/Shape_1"
op_type: "Shape"
attribute {
  name: "start"
  i: 0
  type: INT
}
, input: "/model/model/Shape_1_output_0"
input: "/model/model/transformer/7/dec_attn/Constant_25_output_0"
output: "/model/model/Gather_1_output_0"
name: "/model/model/Gather_1"
op_type: "Gather"
attribute {
  name: "axis"
  i: 0
  type: INT
}
, input: "/model/model/Gather_1_output_0"
input: "/model/Constant_16_output_0"
output: "/model/model/Unsqueeze_2_output_0"
name: "/model/model/Unsqueeze_2"
op_type: "Unsqueeze"
, input: "/model/model/Shape_1_output_0"
input: "/model/model/transformer/Constant_13_output_0"
output: "/model/model/Gather_output_0"
name: "/model/model/Gather"
op_type: "Gather"
attribute {
  name: "axis"
  i: 0
  type: INT
}
, input: "/model/model/Gather_output_0"
input: "/model/Constant_16_output_0"
output: "/model/model/Unsqueeze_1_output_0"
name: "/model/model/Unsqueeze_1"
op_type: "Unsqueeze"
, input: "/model/mod

test 1

In [202]:
import onnxruntime as rt
import numpy as np
import time

sess_options = rt.SessionOptions()

# Set graph optimization level
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

# To enable model serialization after graph optimization set this
sess_options.optimized_model_filepath = "gen.onnx"

session = rt.InferenceSession("gen.onnx", sess_options)
start = time.time()
outputs = session.run(None, {
    "input_ids": ids_tensor.numpy(),
    "n_events": np.array(1000, dtype=np.int64),
    "temperature": np.array(1.2, dtype=np.double),
    "repetition_penalty": np.array(1.2, dtype=np.double),
    "topk": np.array(10, dtype=np.int64),
    })
end = time.time()
print(f"Inference time: {end - start:.4f} seconds")
print(outputs)

2025-05-27 16:58:53.902424 [W:onnxruntime:, graph_utils.cc:142 CanUpdateImplicitInputNameInSubgraphs]  Implicit input name model.model.crit.out_layers.0.weight cannot be safely updated to model.model.transformer.word_emb.emb_layers.0.weight in one of the subgraphs.


Inference time: 15.6901 seconds
[array([[ 230, 4241,  596,  431,  179, 4241, 6383,  487,  192,   58,  456,
        7053,  559,  597,  559,  212, 5185, 2421,  467,  223,   50,  663,
         644,  448,  235, 3949,  528,   58,  519,  183,   56,  456,  755,
         456, 5237,  467,  203, 4831,  209, 5221,  215, 7673, 2824,  456,
         644,  448,  235, 3949,  528,   56,  559, 7270,  467,  190, 3746,
         196, 2791,  202, 3764,  207, 4219,  214, 3192,  219, 3379,  227,
        5713,  233, 3379,  459, 3192,  182, 3680,  755,  410,  194, 2554,
        4916,  461,  207, 2510,  643,  381, 4709,  410,  227, 2510,  232,
        2554,  405, 2510, 1149,  381,  187,   56,  461, 7159,  461,  200,
        2554,  206, 2885,  674,  431,  220, 3922,  225, 4133,  232, 3260,
         405, 4241,  180, 3379,  187, 3192,  194, 3680, 5954,  461, 3123,
         483,  674,  410,  219, 2616,  227, 3764,  234, 2616,  450,   50,
         487,  184, 3764,  190, 2616,  196, 3746,  202, 3521, 5761,  487,
     

In [160]:
input_ids = torch.tensor([1,2,3,4], dtype=torch.int64).unsqueeze(0)  # Dummy input_ids for testing
n_events = torch.tensor(5, dtype=torch.int64)
temperature = torch.tensor(1.0, dtype=torch.float32)
do_sample = torch.tensor(False, dtype=torch.bool)
topk = torch.tensor(10, dtype=torch.int64)
repetition_penalty = torch.tensor(1.0, dtype=torch.float32)
#start timer
import time
start_time = time.time()
gen_seq = gen_wrapper(input_ids, n_events, temperature, repetition_penalty, topk, do_sample)
#end timer
end_time = time.time()
print(f"Generation time: {(end_time - start_time):.4f} seconds")
print(gen_seq)

Generation time: 0.1415 seconds
tensor([[ 404,  264, 4923,  264,  393]])


In [203]:
onnxruntime_extensions.optimize_model(
    "gen.onnx",
    output_file="gen_optimized.onnx",
)



In [205]:
import onnxruntime as rt
import numpy as np
import time

sess_options = rt.SessionOptions()

# Set graph optimization level
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_EXTENDED

# To enable model serialization after graph optimization set this
sess_options.optimized_model_filepath = "gen_optimized.onnx"

session = rt.InferenceSession("gen_optimized.onnx", sess_options)
start = time.time()
outputs = session.run(None, {
    "input_ids": ids_tensor.numpy(),
    "n_events": np.array(1000, dtype=np.int64),
    "temperature": np.array(1.2, dtype=np.double),
    "repetition_penalty": np.array(1.2, dtype=np.double),
    "topk": np.array(10, dtype=np.int64)
    })
end = time.time()
print(f"Inference time: {end - start:.4f} seconds")
print(outputs)

Inference time: 15.4900 seconds
[array([[ 231, 4688,  236,   62,  549,  528, 3088,  184, 3708, 2460,  410,
         195,   67,  512,  200, 5884,  206,   63,  497,  212, 4889, 7195,
         526,  225, 2883,  230, 3717,  507,  387,  179,   58,  497,  184,
        4241,  191, 7034, 1221,  497,  203, 7598,  209,   67,  512,  214,
          65,  410,  220,   63,  483,  226, 6032,  230,   67,  410,  935,
         398,  178,   63,  461,  183,   62,  497,  191, 1569,  195,   62,
         497,  203, 4894,  209, 2207,  214, 3658, 7683,  497,  226,   63,
         461,  232, 7034,  405,   67,  461, 2849,  398,  186,   63,  466,
         192, 7655,  198,   60,  474,  204, 2367,  209, 2961, 6754,  474,
         220, 2746,  225, 2367,  230, 4241, 5951,  474,  234, 2000,  444,
        2207,  181, 3999,  187,   65,  474,  193, 5056,  199, 7034,  799,
         512,  211, 2883,  216, 4167, 5959,  497,  764,  407, 6759,  492,
         444, 2746,  180,   53,  461,  187,   50,  573,  193, 3794,  200,
     